# Continuous Plain MCTS in hand-designed grid

In [ ]:
import wandb
wandb.login()

In [ ]:
import math
import json
from itertools import product
import os
import sys

from hd_grid import hd_grids, number_grids, maze_grids

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils import *
sys.path.append(ROOT_DIR)

from src.Env.Grid.Cont_Grid import Continuous_Grid
from src.Planners.H_MCTS_continuous.Plain_MCTS_cont import Plain_MCTS_Cont

In [ ]:
# Basic Setup for environment
l1_rows = 8
l1_cols = 8
l1_width = 2
l1_height = 2
goal_distance = 3
grid_setting = (l1_rows, l1_cols, l1_width, l1_height, goal_distance)
cont_action_radius=2

In [ ]:
random_seed = 0

In [ ]:
def test_plain_mcts(param, grid_setting, folder_name):
    mcts_result = {}
    mcts_result["iter_cnt"] = {}
    mcts_success_rate = 0

    # # Plain MCTS
    # # Used hyperparameter from best result until now
    # explorationConstant = 0.25
    # alpha = 0.09
    # constant_c = 2
    # gamma = 1
    # iter_Limit = 10000

    explorationConstant = param[0]
    iter_Limit = param[1]
    alpha = param[2]
    constant_c = param[3]
    gamma = param[4]

    for gi in range(len(number_grids)):
        MCTS = Plain_MCTS_Cont(grid_setting, random_seed=random_seed,
                                explorationConstant=1 / math.sqrt(explorationConstant),
                                alpha=alpha,
                                constant_c=constant_c,
                                gamma=gamma,
                                iter_Limit = iter_Limit,
                                assigned_barrier=number_grids[gi].barriers,
                                assigned_start_goal=number_grids[gi].start_goal,
                                cont_action_radius=cont_action_radius,)
        traj, success, iter = MCTS.search("{}/tree/{}.png".format(folder_name, gi))
        iter += 1
        mcts_result["iter_cnt"][gi] = iter

        if success:
            mcts_success_rate += 1
            MCTS.env.plot_grid(0, traj, "{}/traj/{}.png".format(folder_name, gi))
            print(f'success with {iter}')
        else:
            MCTS.env.plot_grid(0, traj, "{}/traj/{}.png".format(folder_name, gi))
            print('Failed')
        
        wandb.log({f"tree_{gi}": wandb.Image("{}/tree/{}.png".format(folder_name, gi))})
        wandb.log({f"traj_{gi}": wandb.Image("{}/traj/{}.png".format(folder_name, gi))})

    mcts_result["success_rate"] = mcts_success_rate
    with open("{}/result.json".format(folder_name), 'w') as result_file:
            json.dump(mcts_result, result_file)
    
    x_values, y_values = cumul_plot(iter_Limit, mcts_result, folder_name)
    wandb.log({"iteration_plot": wandb.Image("{}/success_rate.png".format(folder_name))})
    data = [[x, y] for (x, y) in zip(x_values, y_values)]
    table = wandb.Table(data=data, columns = ["x", "y"])
    wandb.log(
        {"Iteration_Plot" : wandb.plot.line(table, "x", "y",
            title="Iteration vs Success Rate Plot")})
    return mcts_result


In [ ]:
# Launch Experiment
total_runs = 10
for run in range(total_runs):
    # 1. Start a new run to track script
    explorationConstant = [0.16, 0.25, 0.5, 1.0]
    iter_Limit = [10000]
    alpha = [0.01, 0.025, 0.05, 0.09]
    constant_c = [2]
    gamma = [1]

    # Create a list of parameter arrays
    parameters = [explorationConstant, iter_Limit, alpha, constant_c, gamma]

    # Generate all possible combinations
    param_combinations = list(product(*parameters))

    # Print the combinations
    for param in param_combinations:
        print("Param to check", param)

        folder_name = os.path.join(HD_PLAIN_MCTS_EXPERIMENT_DIR, \
                                'wandb_num_grids_test_{}_{}_{}_{}_{}'.format(param[0], param[1], param[2], param[3], param[4]))
        tree_folder_name = os.path.join(folder_name, 'tree')
        traj_folder_name = os.path.join(folder_name, 'traj')

        exists = make_param_dir(folder_name)
        if not exists:
            make_param_dir(tree_folder_name)
            make_param_dir(traj_folder_name)

            wandb.init(
                # Project_name
                project='num-grid',
                # Run_name
                name=f"plain_mcts_{run}_{param[0]}_{param[1]}_{param[2]}_{param[3]}_{param[4]}",
                # Track hyperparameters and run metadata
                config={
                    "explorationConstant": param[0],
                    "iter_Limit": param[1],
                    "alpha": param[2],
                    "constant_c": param[3],
                    "gamma": param[4],
                }
            )
            wandb.run.log_code(".")
            
            mcts_result = test_plain_mcts(param, grid_setting, folder_name)
            wandb.log(mcts_result)
        
        # This simple block simulates a training loop loggin metrics
        wandb.finish()


